In [6]:
%use kandy, multik
import generator.Generator
import generator.Point

In [9]:
import java.text.DecimalFormat

val repeats = 10000
var blocks = 0
repeat(repeats) {
    val (p1, p2) = Generator.nextPariInSquare(h, A)
    val obstacles = HashSet<Point>()
    repeat(poissonDistribution.sample()) {
        obstacles.add(Generator.nextPointInSquare(A))
    }
    var block = false
    check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
        for (p in obstacles) {
            if (i.isInCircle(p, r)) {
                block = true
                break@check
            }
        }
    }
    if (block) {
        ++blocks
    }
}
println("Probability: ${DecimalFormat("#0.00#").format(blocks / repeats.toDouble())}")

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[9], line 6, column 47: Unresolved reference: h
at Cell In[9], line 8, column 12: Unresolved reference: poissonDistribution
at Cell In[9], line 14, column 33: Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public val Int.r: RInt defined in org.jetbrains.kotlinx.multik.ndarray.data

In [7]:
//fun mathModel(a: Number, h: Number, r: Number): Double {
//    return min(2 * r.toDouble() * h.toDouble() * a.toDouble(), 1.0)
//}
fun mathModel(a: Number, h: Number, r: Number): Double {
    return 1 - exp(-a.toDouble() * h.toDouble() * 2 * r.toDouble())
}

In [8]:
import org.apache.commons.math3.distribution.PoissonDistribution
import java.text.DecimalFormat

val repeats = 1000

val A = 10 // square side

val hs = listOf<Number>(1, 4, 8) // point distance
val `as` = listOf<Number>(0.01, 0.05, 0.1, 0.3) // obstacle density
val rs = listOf<Number>(0.05, 0.1, 0.5) // obstacle size
val df = DecimalFormat("#0.00#")

println("a\th\tr\tImitation\tModel\tCV")
var cvsum = 0.0
var cvcount = 0
for (a in `as`) {
    val poissonDistribution = PoissonDistribution(a.toDouble() * A * A)
    for (h in hs) {
        for (r in rs) {
            var blocks = 0
            repeat(repeats) {
                val (p1, p2) = Generator.nextPariInSquare(h, A)
                val obstacles = HashSet<Point>()
                repeat(poissonDistribution.sample()) {
                    obstacles.add(Generator.nextPointInSquare(A))
                }
                var block = false
                check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
                    for (p in obstacles) {
                        if (i.isInCircle(p, r)) {
                            block = true
                            break@check
                        }
                    }
                }
                if (block) {
                    ++blocks
                }
            }
            val imitation = blocks / repeats.toDouble()
            val mathModel = mathModel(a, h, r)
            val diff = imitation - mathModel
            val cv = diff.pow(2) / (abs(if (diff == 0.0) 1.0 else diff) / 2)
            cvsum += cv
            ++cvcount
            println("$a\t$h\t$r\t${df.format(imitation)}\t${df.format(mathModel)}\t${df.format(cv)}")
        }
    }
}
println("cv mean = ${df.format(cvsum / cvcount)}")

a	h	r	Imitation	Model	CV
0.01	1	0.05	0.001	0.001	0.00
0.01	1	0.1	0.004	0.002	0.004
0.01	1	0.5	0.019	0.01	0.018
0.01	4	0.05	0.005	0.004	0.002
0.01	4	0.1	0.004	0.008	0.008
0.01	4	0.5	0.032	0.039	0.014
0.01	8	0.05	0.007	0.008	0.002
0.01	8	0.1	0.014	0.016	0.004
0.01	8	0.5	0.083	0.077	0.012
0.05	1	0.05	0.006	0.005	0.002
0.05	1	0.1	0.015	0.01	0.01
0.05	1	0.5	0.08	0.049	0.062
0.05	4	0.05	0.021	0.02	0.002
0.05	4	0.1	0.038	0.039	0.002
0.05	4	0.5	0.204	0.181	0.045
0.05	8	0.05	0.041	0.039	0.004
0.05	8	0.1	0.059	0.077	0.036
0.05	8	0.5	0.335	0.33	0.011
0.1	1	0.05	0.013	0.01	0.006
0.1	1	0.1	0.022	0.02	0.004
0.1	1	0.5	0.148	0.095	0.106
0.1	4	0.05	0.041	0.039	0.004
0.1	4	0.1	0.091	0.077	0.028
0.1	4	0.5	0.383	0.33	0.107
0.1	8	0.05	0.09	0.077	0.026
0.1	8	0.1	0.139	0.148	0.018
0.1	8	0.5	0.576	0.551	0.051
0.3	1	0.05	0.022	0.03	0.015
0.3	1	0.1	0.066	0.058	0.016
0.3	1	0.5	0.396	0.259	0.274
0.3	4	0.05	0.114	0.113	0.002
0.3	4	0.1	0.216	0.213	0.005
0.3	4	0.5	0.768	0.699	0.138
0.3	8	0.05	0.192	0.213	0.043
0.3	8

In [10]:
import org.apache.commons.math3.distribution.PoissonDistribution

val A = 10 // square side

val h = 3 // point distance
val a = 0.2 // obstacle density
val r = 0.1 // obstacle size

val poissonDistribution = PoissonDistribution(a * A * A)

val (p1, p2) = Generator.nextPariInSquare(h, A)
val obstacles = HashSet<Point>()
repeat(poissonDistribution.sample()) {
    obstacles.add(Generator.nextPointInSquare(A))
}

val psize = 25
plot {
    
    points { 
        x(obstacles.map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(obstacles.map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        size = psize.toDouble() * r
    }

    points {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        color = Color.RED
    }

    line {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        type = LineType.DASHED
        color = Color.RED
    }

    layout {
        size = Pair(A * psize, A * psize)
        style { 
            axis {
                blankAxes()
            }
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="lRyZaQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":250.0,
"height":250.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[6.355508692784925,5.03037779168042,3.728217583898059,3.923698545326706,9.294314610073855,3.8579122768482788,1.218420124371955,0.9589769514669821,1.628526728064743,8.13169558854667,9.832339471790362,6.867156356054017,5.01101599315321,2.0491802563267836,7.631868461400311],
"y":[4.77410898191443,6.090653234566117,9.166883878768033,6.427503650908827,2.280292587850954,5.458521629770262,9.869208797508572,9.521608202014086,6.124077415034491,9.95045559674754,1.0113904387873263,3.2902491456827097,0.7727509753064932,6.399816223539187,1.832325388792999]
},
"size":2.5,
"sampling":"none",
"position":"identity",
"geom":"point"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[2.5512976891217534,5.325744541664689],
"y":[5.7503101057546715,6.891557082712155]
},
"color":"#ee6666",
"sampling":"none",
"position":"identity",
"geom":"point"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[2.5512976891217534,5.325744541664689],
"y":[5.7503101057546715,6.891557082712155]
},
"color":"#ee6666",
"linetype":"dashed",
"sampling":"none",
"position":"identity",
"geom":"line"
}],
"theme":{
"axis":"blank",
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("lRyZaQ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>">

In [11]:
import org.apache.commons.math3.distribution.PoissonDistribution
import org.jetbrains.kotlinx.dataframe.api.dataFrameOf
import java.text.DecimalFormat

val repeats = 1000

val A = 10 // square side

//val hs = (1..100 step 1).map { it / 10.0 } // point distance
//val `as` = listOf(0.1) // obstacle density
//val rs = listOf(0.1) // obstacle size

val hs = listOf(3) // point distance
val `as` = listOf(0.1) // obstacle density
val rs = (1..1000 step 10).map { it / 1000.0 } // obstacle size

val ps = ArrayList<Double>()

for (a in `as`) {
    val poissonDistribution = PoissonDistribution(a.toDouble() * A * A)
    for (h in hs) {
        for (r in rs) {
            var blocks = 0
            repeat(repeats) {
                val (p1, p2) = Generator.nextPariInSquare(h, A)
                val obstacles = HashSet<Point>()
                repeat(poissonDistribution.sample()) {
                    obstacles.add(Generator.nextPointInSquare(A))
                }
                var block = false
                check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
                    for (p in obstacles) {
                        if (i.isInCircle(p, r)) {
                            block = true
                            break@check
                        }
                    }
                }
                if (block) {
                    ++blocks
                }
            }
            ps.add(blocks / repeats.toDouble())
        }
    }
}

dataFrameOf(
    "r" to `rs`.toList(),
    "p" to ps
).plot { 
    line {
        x("r")
        y("p")
    }
    layout {
        size = Pair(2 * A * psize, A * psize)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="Y4RGH1"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"p":[0.0,0.008,0.011,0.02,0.021,0.031,0.026,0.029,0.057,0.05,0.075,0.072,0.083,0.066,0.086,0.102,0.097,0.116,0.104,0.103,0.112,0.135,0.138,0.152,0.133,0.177,0.172,0.165,0.166,0.17,0.205,0.198,0.188,0.191,0.186,0.263,0.207,0.211,0.215,0.247,0.259,0.257,0.253,0.267,0.279,0.304,0.288,0.297,0.312,0.328,0.313,0.3,0.324,0.334,0.333,0.321,0.35,0.35,0.345,0.364,0.366,0.387,0.415,0.376,0.411,0.421,0.402,0.442,0.398,0.44,0.427,0.398,0.414,0.459,0.437,0.437,0.456,0.469,0.474,0.477,0.448,0.5,0.533,0.523,0.507,0.506,0.504,0.501,0.52,0.531,0.528,0.555,0.572,0.544,0.542,0.581,0.567,0.596,0.576,0.587],
"r":[0.001,0.011,0.021,0.031,0.041,0.051,0.061,0.071,0.081,0.091,0.101,0.111,0.121,0.131,0.141,0.151,0.161,0.171,0.181,0.191,0.201,0.211,0.221,0.231,0.241,0.251,0.261,0.271,0.281,0.291,0.301,0.311,0.321,0.331,0.341,0.351,0.361,0.371,0.381,0.391,0.401,0.411,0.421,0.431,0.441,0.451,0.461,0.471,0.481,0.491,0.501,0.511,0.521,0.531,0.541,0.551,0.561,0.571,0.581,0.591,0.601,0.611,0.621,0.631,0.641,0.651,0.661,0.671,0.681,0.691,0.701,0.711,0.721,0.731,0.741,0.751,0.761,0.771,0.781,0.791,0.801,0.811,0.821,0.831,0.841,0.851,0.861,0.871,0.881,0.891,0.901,0.911,0.921,0.931,0.941,0.951,0.961,0.971,0.981,0.991]
},
"ggsize":{
"width":500.0,
"height":250.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"r",
"y":"p"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("Y4RGH1");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M19.862059035441753 182.3181818181818 L19.862059035441753 182.3181818181818 L23.874596214318874 179.9874923733984 L27.887133393195995 179.11348383160464 L31.89967057207312 176.4914582062233 L35.912207750950245 176.20012202562538 L39.92474492982736 173.2867602196461 L43.93728210870448 174.74344112263574 L47.949819287581604 173.86943258084196 L51.96235646645873 165.71201952410007 L55.97489364533585 167.75137278828555 L59.98743082421297 160.4679682733374 L63.999968003090096 161.34197681513118 L68.01250518196721 158.137278828554 L72.02504236084434 163.08999389871872 L76.03757953972146 157.26327028676022 L80.05011671859859 152.6018913971934 L84.06265389747571 154.05857230018304 L88.07519107635284 148.52318486882245 L92.08772825522993 152.01921903599757 L96.10026543410706 152.31055521659547 L100.11280261298418 149.68852959121415 L104.12533979186131 142.98779743746186 L108.13787697073843 142.11378889566808 L112.15041414961556 138.03508236729715 L116.16295132849268 143.5704697986577 L120.1754885073698 130.751677852349 L124.18802568624693 132.20835875533862 L128.20056286512406 134.2477120195241 L132.21310004400118 133.95637583892616 L136.22563722287828 132.79103111653447 L140.2381744017554 122.59426479560707 L144.25071158063253 124.63361805979255 L148.26324875950965 127.54697986577182 L152.27578593838678 126.67297132397803 L156.2883231172639 128.12965222696766 L160.30086029614102 105.6967663209274 L164.31339747501815 122.01159243441123 L168.32593465389527 120.84624771201953 L172.3384718327724 119.68090298962782 L176.35100901164952 110.3581452104942 L180.36354619052665 106.8621110433191 L184.37608336940374 107.44478340451495 L188.38862054828087 108.61012812690666 L192.401157727158 104.53142159853569 L196.41369490603512 101.03538743136058 L200.42623208491224 93.75198291641246 L204.43876926378937 98.41336180597926 L208.45130644266

$P_{\text{loss}} = 1 - e^{\lambda S}$

$S = h * 2r$